# BATCHNORM2D
4D数据上应用batch归一化。减少内部协方差偏移（Reducing Internal Covariate Shift，其实是就是输入数据与输出数据分布发生变化，如传统机器学习中训练集训练的模型，随着数据变化产生的测试集，二者就存在了斜方差偏移，而神经网络则是随着层数增加，训练过程中会导致内部层之间数据分布变化），加速训练。

$$
y = \frac{x - E[x]}{\sqrt{Var[x] + \epsilon }} * \gamma + \beta
$$

均值与标准差都是Min-Batch维度的，$\gamma $,$\beta$是大小为C（输入数据大小）的可学习参数。函数参数affine=False时，$\gamma = 1$,$\beta = 0$

默认情况下，训练这一层时会保留这一层计算的mean与variance，在评估时用于归一化。使用默认的momentum=0.1。

**BatchNormalization在激活函数之前生效,其中running_mean,running_var都是在forward中更新**

[SOURCE CODE FOR TORCH.NN.MODULES.BATCHNORM][3]

## 参数
- num_features: C,来自输入数据(N,C,H,W)，**通道数量**
- eps:分子中的$\epsilon$，默认值为$10^{-5}$
- affine(仿射):boolean，默认值为True，需要训练$\gamma$, $\beta$
- track_running_stats:boolean,默认值为True，跟踪运行时mean与variance，如果设置为False，不记录这些统计信息，并且将running_mean与running_var为None，则train与eval时仅仅使用batch statistics。
- momentum：用于计算running_mean与running_var。

tips:
> 此momentum不同于optimizer中与传统的momentum。数学的讲，此处running statistics的更新规则：
> 
> $\hat{x}_{new} = (1 - momentum) * \hat{x} + mometum * x_t$，其中$\hat{x}$是estimated statistic，$x_t$是新的observed value。

# 训练与推断时使用
训练时，每次都是min-batch数据，得到的均值与方差也是batch级别的

推断时，此时只有一个样本，此时的均值方差使用全局的（所有数据在本层的均值与方差）。

# BN带来的好处
1. 不需要那么小心的初始化参数了
2. 加速训练（如SGD寻找最优点时不会大幅跳动）
3. 一定程度缓解过拟合（不能阻止）与梯度消失，提高了泛化能力：诡异话后的数据一定程度使搜索边界更加平滑。
4. 最初目的：解决内部协方差偏移

tips:
> batch过小的话，均值与var波动较大，不稳定，影响效果。

# model.eval()与model.train()
在包含BN与Dropout（Batch训练时，支持按概率忽略部分特征检测器-目前用BN较多时有效。

调用`train()`(默认参数为True)，启用BN或Dropout，BN中记录batch与全局的mean与var进行更新。如果在test模式中启用，则会基于该样本计算与更新全局与batch的mean与var（他们就在forward中更新），导致结果异常。

调用`eval()`(实际上调用`train(False)`),不启用BN或Dropout。推断时，直接使用全局的mean与var


# Ref
[Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167)

https://www.zhihu.com/question/38102762

[Batch Normalization，批规范化](https://blog.csdn.net/LoseInVain/article/details/86476010):里面有很好的参数冻结的例子与BN参数设置

[深度学习之Batch Normalization
](https://www.cnblogs.com/Luv-GEM/p/10756382.html):协变量偏移问题的产生

[SOURCE CODE FOR TORCH.NN.MODULES.BATCHNORM](https://pytorch.org/docs/stable/_modules/torch/nn/modules/batchnorm.html#BatchNorm2d)：BN代码

[1]: https://pytorch.org/docs/stable/_modules/torch/nn/modules/batchnorm.html#BatchNorm2d    "SOURCE CODE FOR TORCH.NN.MODULES.BATCHNORM"

[Module.eval](https://pytorch.org/docs/stable/_modules/torch/nn/modules/module.html#Module.eval)：eval代码